In [1]:
import os

import tensorflow as tf
import numpy as npo

import autograd.numpy as np
from autograd import grad
from autograd.scipy.integrate import odeint
from autograd.builtins import tuple
from autograd.misc.optimizers import adam

# from scipy.integrate import odeint
# from scipy.interpolate import interp1d

import time
from tqdm import tqdm

In [3]:
nPat        = 1
Nnt         = 3
fj          = np.hstack([np.array([1.2 , 0.7, 1.5 ])] * nPat).reshape(nPat, -1).astype(np.float32)
rj          = np.hstack([np.array([0.6 , 0.3, 0.8 ])] * nPat).reshape(nPat, -1).astype(np.float32)
mj          = np.hstack([np.array([1.0 , 1.7, 0.2 ])] * nPat).reshape(nPat, -1).astype(np.float32)
Taus        = np.array([ 1, 1, 1 ] * nPat).reshape(nPat, -1).astype(np.float32)

NNwts  = [ np.random.rand( 6, 12), 
           np.random.rand(12,  3),
           np.random.rand( 3,  3) ]
NNb    = [ 0.0, 1.0, -1.0 ]
NNact  = [ np.tanh, np.tanh, np.tanh ]

stress_m    = np.hstack([np.array([2.1 , 3.5, 2.4])] * nPat).reshape(nPat, -1).astype(np.float32)

def rhs(y, t, params):

    fj, rj, mj, NNwts, NNb, Taus = params

    y         = np.array(y).reshape(nPat, -1)
    Nnt_val   = y[:, :Nnt]

    nn_inputs  = np.concatenate([Nnt_val, stress_m], axis=1)
    for index, (w, b, a) in enumerate(zip(NNwts, NNb, NNact)):
        nn_res    = np.dot(nn_inputs, w) + b
        nn_res    = a(nn_res)
        nn_inputs = nn_res

    nn_res    = nn_res - y[:, Nnt:] / Taus

    meds_res  = fj - rj * Nnt_val - mj * Nnt_val
    result    = np.concatenate([nn_res, meds_res], axis=1) / 100
    result    = result.flatten()

    return result

params    = [fj, rj, mj, NNwts, NNb, Taus]

start     = time.time()
true_y    = odeint(rhs, y0=np.array([1, 1, 1, 2, 2, 2] * nPat), t=np.linspace(0, 100, 101), args=(params,))
ODEcost   = time.time() - start
print('N', nPat, 'ODEcost', ODEcost, 'perPax', ODEcost / nPat)
print(true_y.min(), true_y.max())

print(rhs(y=np.array([1, 1, 1, 2, 2, 2] * nPat), t=np.linspace(0, 100, 101), params=params))

N 1 ODEcost 0.0055048465728759766 perPax 0.0055048465728759766
-0.978799834464 3.30398774529
[-0.01628093 -0.02005862 -0.01319117 -0.004      -0.013       0.005     ]


In [4]:
def loss(params, iterations):
    pred_y   = odeint(rhs, np.array([1, 1, 1, 2, 2, 2] * nPat), np.linspace(0, 100, 101), tuple((params,)))
    return np.abs(true_y - pred_y).mean()

init_params = [ np.hstack([np.zeros(shape=(3,))] * nPat).reshape(nPat, -1).astype(np.float32), 
                np.hstack([np.zeros(shape=(3,))] * nPat).reshape(nPat, -1).astype(np.float32),
                np.hstack([np.zeros(shape=(3,))] * nPat).reshape(nPat, -1).astype(np.float32),
                [ np.zeros(shape=(6, 12)), 
                  np.zeros(shape=(12,  3)),
                  np.zeros(shape=( 3,  3)) ],
               [ 0.0, 0.0, 0.0 ],
               np.array([ 1, 1, 1 ] * nPat).reshape(nPat, -1).astype(np.float32)
              ]

In [5]:
gradLoss = grad(loss)
gradLoss(init_params, 0)

[array([[-0.03904702,  0.02659655, -0.05541665]], dtype=float32),
 array([[ 0.00067873, -0.02985626,  0.01361074]], dtype=float32),
 array([[ 0.00067873, -0.02985626,  0.01361074]], dtype=float32),
 [array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
        

In [6]:
pbar   = tqdm(range(10))

def callback(params, iterations, g):

    pred_y      = odeint(rhs, np.array([1, 1, 1, 2, 2, 2] * nPat), np.linspace(0, 100, 101), tuple((params,)))
    trainLoss   = np.abs(true_y - pred_y).mean()
    description = "Iteration {:d} train loss {:.6f}".format(
                      iterations, trainLoss)
    pbar.set_description(description)
    pbar.update(1)

start     = time.time()
optimized_params = adam(grad(loss), init_params, num_iters=10, callback=callback)
TrainCost = time.time() - start
print('N', nPat, 'TrainCost', TrainCost, 'perPax', TrainCost / nPat)

Iteration 9 train loss 0.214783: 100%|██████████| 10/10 [00:50<00:00,  5.05s/it]

N 1 TrainCost 50.25134801864624 perPax 50.25134801864624
